# Web Scraping for Indeed.com & Predicting Salaries

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary predictor with Logistic Regression.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use Logistic Regression.

- Question: Why would we want this to be a classification problem?
- Answer: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

In [1]:
URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"


In [2]:
import requests
import bs4
from bs4 import BeautifulSoup
from __future__ import division, print_function

import requests
from bs4 import BeautifulSoup
from selenium import webdriver

import pandas as pd
import numpy as np

from IPython.core.display import HTML, Image

In [3]:
r = requests.get(URL)
indeed = BeautifulSoup(r.content, "lxml")


In [4]:
col_list = ['location', 'title', 'company', 'salary', 'description']

def prime_time(soup):
    location_list = []
    role_title_list = []
    company_list = []
    salaries = []
    summaries = []

    for t in soup.findAll("div", class_='result'):
#     url = t.find('a', 
#     url_list.append(url) #---- can return to this later
        location = t.find('span', class_='location').text # WORKS!
        location_list.append(location)
        title = t.find('a')['title'] # WORKS!
        role_title_list.append(title)
        company_name = t.find('span', class_='company').text
        company_name = company_name.strip('\n')
        company_list.append(company_name)
        salary = t.find('nobr' or 'sjcl')   #('nobr').next_sibling sjcl
        salaries.append(salary)
        summary = t.find('span', class_="summary").text
        summaries.append(summary)
    
    results = zip(location_list, role_title_list, company_list, salaries, summaries)
    results = pd.DataFrame(results)
    results.columns = col_list
    return(results)

# indeed_results = prime_time(indeed)
# indeed_output = pd.DataFrame(indeed_results)
# indeed_output.head()



Let's look at one result more closely. A single `result` looks like

```
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=2480d203f7e97210&amp;jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a `nobr` element inside of a `td` element with `class='snip`.
- The title of a job is in a link with class set to `jobtitle` and a `data-tn-element="jobTitle`.  
- The location is set in a `span` with `class='location'`. 
- The company is set in a `span` with `class='company'`. 

### Write 4 functions to extract each item: location, company, job, and salary.

example: 
```python
def extract_location_from_result(result):
    return result.find ...
```


- Make sure these functions are robust and can handle cases where the data/field may not be available.
- Test the functions on the results above
- Include any other features you may want to use later (e.g. summary, #of reviews...)

Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

### Complete the following code to collect results from multiple cities and starting points. 
- Indeed.com only has salary information for an estimated 20% of job postings. You may want to add other cities to the list below to gather more data. 
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

In [5]:
import numpy as np

url_template_first = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}"
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"

max_results_per_city = 100
col_list = ['location', 'title', 'company', 'salary', 'description']
sample_vals = [np.NaN,np.NaN,np.NaN,np.NaN,np.NaN]

d = dict.fromkeys(col_list, sample_vals)
results = pd.DataFrame(d)   # trying to make the columns the same so I can merge later


for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Atlanta','Boston','Berkeley',\
                 'Oakland', 'Raleigh', 'Brooklyn','Queens', 'St+Louis','Asheville','Portland','Maine',\
                 'Baltimore', 'Boulder', 'Denver', 'Reno', 'Des Moines', 'Cleveland',  'Detroit', 'Kansas+City',\
                 'Dallas', 'Washington+DC', 'Nashville', 'Seattle', 'Salt+Lake+City', 'Sacramento', 'Minneapolis', 'St+Paul']): 
    for start in range(0, max_results_per_city, 10):
        foo = requests.get(url_template.format(city, start))
        indeed = BeautifulSoup(foo.content, "lxml")
        indeed_results = prime_time(indeed)
        indeed_results.columns = col_list
        
        results = pd.concat([indeed_results, results], ignore_index=True)
    first = requests.get(url_template_first.format(city)) #GETS THE FIRST PAGE OF RESULTS, MISSED
    first_indeed = BeautifulSoup(first.content, "lxml")
    first_indeed_results = prime_time(first_indeed)
    first_indeed_results.columns = col_list
    results = pd.concat([first_indeed_results, results], ignore_index=True)
results.head(50)



,company,description,location,salary,title
0,Maine Health Management Coalition,Understands mapping of data elements to analyt...,"Topsham, ME",None,Senior Research Analyst
1,"Sensory, Inc","16-01 - Research Scientist – BOULDER, CO. As a...",United States,None,Research Scientist [Exp. with Machine Learning...
2,U.S. Air Force,Help Airman get the proper healthcare they nee...,United States,None,Biomedical Laboratory Officer
3,Etech Hi Inc.,"\nData analyst, data programmer, data scientis...","Brewer, ME","<nobr>$120,000 a year</nobr>",Data Analyst II/III
4,Xpandit,"\nBut, more than just a “Buzzword Guy”, we see...","Lisbon, ME",None,Big Data Engineer
5,Maine Health Management Coalition,\nUnderstands mapping of data elements to anal...,"Topsham, ME",None,Senior Research Analyst
6,Alere,\nAdvanced knowledge of a variety of statistic...,"Scarborough, ME",None,Statistician
7,Alere,\nConducts and interprets data. Phases of a pr...,"Scarborough, ME",None,Scientist I (Research Assoc)
8,IDEXX Laboratories,"\nPrepares materials, performs experiments, ga...","Westbrook, ME 04092",None,Associate Scientist I
9,IDEXX Laboratories,\nWill interpret internally and externally gen...,"Westbrook, ME 04092",None,Systems Engineer


#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

Lastly, we need to clean up salary data. 
1. Some of the salaries are not yearly but hourly, these will be useful to us for now 2. The salaries are given as text and usually with ranges.

#### Filter out the salaries that are not yearly (filter those that refer to hour)

In [6]:
import numpy as np


            
            
# results['annual_salary'] = results['salary']

results_nan = results.replace({None:np.NaN})
results = results_nan
results.head(10)
            

,company,description,location,salary,title
0,Maine Health Management Coalition,Understands mapping of data elements to analyt...,"Topsham, ME",NaN,Senior Research Analyst
1,"Sensory, Inc","16-01 - Research Scientist – BOULDER, CO. As a...",United States,NaN,Research Scientist [Exp. with Machine Learning...
2,U.S. Air Force,Help Airman get the proper healthcare they nee...,United States,NaN,Biomedical Laboratory Officer
3,Etech Hi Inc.,"\nData analyst, data programmer, data scientis...","Brewer, ME","<nobr>$120,000 a year</nobr>",Data Analyst II/III
4,Xpandit,"\nBut, more than just a “Buzzword Guy”, we see...","Lisbon, ME",NaN,Big Data Engineer
5,Maine Health Management Coalition,\nUnderstands mapping of data elements to anal...,"Topsham, ME",NaN,Senior Research Analyst
6,Alere,\nAdvanced knowledge of a variety of statistic...,"Scarborough, ME",NaN,Statistician
7,Alere,\nConducts and interprets data. Phases of a pr...,"Scarborough, ME",NaN,Scientist I (Research Assoc)
8,IDEXX Laboratories,"\nPrepares materials, performs experiments, ga...","Westbrook, ME 04092",NaN,Associate Scientist I
9,IDEXX Laboratories,\nWill interpret internally and externally gen...,"Westbrook, ME 04092",NaN,Systems Engineer


In [7]:
def salary_clean(string):
    if type(string) == float:
        return np.NaN
    elif string == np.NaN:
        return np.NaN
    else:
        try:
            return((string.split("<nobr>")[1]).split("</nobr>")[0])
        except (TypeError, IndexError): 
            return np.NaN

for b in range(len(results['salary'])):
    results['salary'][b] = salary_clean(str(results['salary'][b]))
    

results.head(15)

,company,description,location,salary,title
0,Maine Health Management Coalition,Understands mapping of data elements to analyt...,"Topsham, ME",NaN,Senior Research Analyst
1,"Sensory, Inc","16-01 - Research Scientist – BOULDER, CO. As a...",United States,NaN,Research Scientist [Exp. with Machine Learning...
2,U.S. Air Force,Help Airman get the proper healthcare they nee...,United States,NaN,Biomedical Laboratory Officer
3,Etech Hi Inc.,"\nData analyst, data programmer, data scientis...","Brewer, ME","$120,000 a year",Data Analyst II/III
4,Xpandit,"\nBut, more than just a “Buzzword Guy”, we see...","Lisbon, ME",NaN,Big Data Engineer
5,Maine Health Management Coalition,\nUnderstands mapping of data elements to anal...,"Topsham, ME",NaN,Senior Research Analyst
6,Alere,\nAdvanced knowledge of a variety of statistic...,"Scarborough, ME",NaN,Statistician
7,Alere,\nConducts and interprets data. Phases of a pr...,"Scarborough, ME",NaN,Scientist I (Research Assoc)
8,IDEXX Laboratories,"\nPrepares materials, performs experiments, ga...","Westbrook, ME 04092",NaN,Associate Scientist I
9,IDEXX Laboratories,\nWill interpret internally and externally gen...,"Westbrook, ME 04092",NaN,Systems Engineer


In [8]:
# print(results['salary'].isnull().value_counts()) #only 62 salaries in here it appears


#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [9]:
import re
import string 

def salarizer(salary):  
    if pd.isnull(salary) == False:
        salary = salary.translate(None, "$|anmonthhurye, ")
        return(salary) # test line -- cut interior function to split/ average and moved outside
    else:
        return np.NaN

    


def salary_average(salary): 
    float_sal_list = []
    try:
        salary_str = str(salary)
        sal_list = str.split(salary_str, "-")
        for item in sal_list:
            try:
                float_sal_list.append(float(item))
            except ValueError:
                return salary
        if len(float_sal_list) > 1:
            avg_sal = sum(float_sal_list)/float(len(float_sal_list))
            return(avg_sal)
        else:
            return float(salary)
    except IndexError:
        return np.NaN
    

    
for hannah in range(len(results['salary'])):
    results['salary'][hannah] = salarizer(results['salary'][hannah])
    results['salary'][hannah] = salary_average(results['salary'][hannah])
results.head(20)        
    

,company,description,location,salary,title
0,Maine Health Management Coalition,Understands mapping of data elements to analyt...,"Topsham, ME",NaN,Senior Research Analyst
1,"Sensory, Inc","16-01 - Research Scientist – BOULDER, CO. As a...",United States,NaN,Research Scientist [Exp. with Machine Learning...
2,U.S. Air Force,Help Airman get the proper healthcare they nee...,United States,NaN,Biomedical Laboratory Officer
3,Etech Hi Inc.,"\nData analyst, data programmer, data scientis...","Brewer, ME",120000,Data Analyst II/III
4,Xpandit,"\nBut, more than just a “Buzzword Guy”, we see...","Lisbon, ME",NaN,Big Data Engineer
5,Maine Health Management Coalition,\nUnderstands mapping of data elements to anal...,"Topsham, ME",NaN,Senior Research Analyst
6,Alere,\nAdvanced knowledge of a variety of statistic...,"Scarborough, ME",NaN,Statistician
7,Alere,\nConducts and interprets data. Phases of a pr...,"Scarborough, ME",NaN,Scientist I (Research Assoc)
8,IDEXX Laboratories,"\nPrepares materials, performs experiments, ga...","Westbrook, ME 04092",NaN,Associate Scientist I
9,IDEXX Laboratories,\nWill interpret internally and externally gen...,"Westbrook, ME 04092",NaN,Systems Engineer


In [10]:
#check if salary_average works

salary = results['salary'][10]
# salary = re.sub('$|anmonthhur,', '', str(salary))
salary = salary_average(salary)
print(salary)



46332.0


In [ ]:
#check if all my values are floats

da_list = []
for gabe in results['salary']:
    if pd.isnull(gabe) == False:
        da_list.append(gabe)
        
print(da_list)

[120000.0, 46332.0, 49628.5, 82964.5, 89405.5, 82964.5, 89405.5, 82964.5, 89405.5, 82964.5, 89405.5, 82964.5, 89405.5, 82964.5, 89405.5, 82964.5, 89405.5, 82964.5, 89405.5, 82964.5, 89405.5, 120000.0, 46332.0, 49628.5, 110000.0, 120000.0, 5438.5, 165000.0, 100000.0, 165000.0, 160000.0, 84649.0, 137500.0, 32.31, 180000.0, 110000.0, 120000.0, 78195.5, '95d', 5790.0, 5790.0, 2864.0, 3306.5, 2864.0, 25.24, 3306.5, 32.215, 2864.0, 60000.0, 15.0, 42840.0, 15.0, 78195.5, 25.0, 15.0, 155000.0, 31.189999999999998, 76856.5, 110000.0, 43014.5, 103014.0, 22.825, 38.365, 31.189999999999998, 76856.5, 127500.0, 110000.0, 240000.0, 130000.0, 112500.0, 150000.0, 90000.0, 100000.0, 100000.0, 127500.0, 110000.0, 16.0, 16.0, 140000.0, 17.17, 140000.0, 23.36, 68216.0, 110000.0, 28.525, 30000.0, 190000.0, 58175.5, 50112.0, 80000.0, 78483.0, 54351.5, 140000.0, 65643.5, 50508.0, 67373.5, 46332.0, 107500.0, 78483.0, 43123.5, 48201.0, 60934.5, 28.525, 30000.0, 190000.0, 58175.5, 50112.0, 165000.0, 130000.0, 132

In [ ]:
#categorize hourly, monthly, salaries
results['hourly_salary'] = np.NaN
results['monthly_salary'] = np.NaN
results['annual_salary'] = np.NaN

for where, number in enumerate(results['salary']):
    if number < 200:
        results['hourly_salary'][where] = number
    elif number <= 10000: #check if this is fair, based on 8.5k type figures
        results['monthly_salary'][where] = number
    else:
        results['annual_salary'][where] = number
        
results.head(50)

### Save your results as a CSV

In [ ]:
#here I should remove duplicates!!! Addition of Oakland, Berkeley, SF etc... will lead to dupes

results.drop_duplicates(inplace=1)

In [ ]:
results.to_csv("indeed_ds_data30_dedupe.csv", encoding='utf8')